In [8]:
from models.GPT import GPTLanguageModel
import os
import pickle
import torch

In [9]:
if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [10]:
with open("./data/wizard of oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [11]:
from tokenization.tokenizer import BasicTokenizer

tokenizer = BasicTokenizer()
tokenizer.build_vocab(chars)

data = torch.tensor(tokenizer.encode(text), dtype=torch.long)

In [12]:
from utils.helper import train_val_test_split

train_data, val_data, test_data = train_val_test_split(data, test_size=0.2, val_size=0.2)

In [13]:
from utils.helper import load_model, save_model
import os

model = GPTLanguageModel(vocab_size=vocab_size, device=device)

model_name = "model-01.pk1"
model_path = os.path.join(os.path.abspath(os.getcwd()), model_name)

if os.path.exists(model_path):
    model = load_model(model_path)

m = model.to(device)

Loading parameters...
Model loaded successfully!


In [14]:
model.set_train_data(train_data)
model.set_val_data(val_data)

model._train(epochs=200, learning_rate=3e-4, eval_iters=100)
save_model(model_path, model)

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0, train_loss: 8.207, val_loss: 8.220


KeyboardInterrupt: 

In [ ]:
while True:
    prompt = input("Your prompt: ")
    context = torch.tensor(tokenizer.encode(prompt), dtype=torch.long, device=device)
    generated_chars = tokenizer.decode(m.generate(context.unsqueeze(0), max_new_tokens=150)[0].tolist())
    print(generated_chars)